# Calculate synthetic arrival times using a Python script

`Cake` allows to calculate the arrival times of different seismic phases ([example here](https://pyrocko.org/docs/current/apps/cake/manual.html#calculate-p-phase-arrivals)). In this task we will compute P and S wave arrivals for the station `GE.KTHA` and the 2020 M<sub>w</sub> 7.0 Samos earthquake and compare them to the picks, we made in the previous module.

We start by importing the different modules from Pyrocko, we need:

In [ ]:
from pyrocko import cake, model, util
from pyrocko.guts import clone
from pyrocko.gui.util import EventMarker

import numpy as num

In the next step both the event and station data need to be loaded. We make use of `load_one_event` and `load_stations` for this purpose.

In [ ]:
# Loading of events
event = model.load_one_event(filename='data/events.yaml')

# Loading of stations
stations = model.load_stations(filename='data/stations.yaml')

In the next step we need to select the station `GE.KTHA`:

In [ ]:
station = None

# Iterate through stations and check for matching network-station-location code
for sta in stations:
    if sta.nsl() == ('GE', 'KTHA', ''):
        station = sta
        
        break

and calculate the distance to the earthquake:

In [ ]:
distance = event.distance_to(station) * cake.m2d  # Convert meter to degrees for usage in cake

print('Distance between event and station: {:.4f} deg'.format(distance))

Great, finally we can calculate the arrival times for the different phases based on **cake**. First the synthetic earth model is loaded:

In [ ]:
# Load builtin 'prem-no-ocean' model (medium resolution)
earth_model = cake.load_model('prem-no-ocean.m')

We also need to define the phases, we want to extract the travel time for:

In [ ]:
# Define the phase to use.
phases = [
    cake.PhaseDef('P'),
    cake.PhaseDef('S')]

Now arrivals are calulated and we directly feed it into `Snuffler` markers for usage in `Snuffler`. 

In [ ]:
markers = []

# calculate distances and arrivals and store them as marker:
print('{:<12}|{:>11}'.format('phase name', 'arrival'))
print('-' * 24)

for arrival in earth_model.arrivals([distance], phases=phases, zstart=event.depth):
    phase = arrival.used_phase()
    
    # Create pseudo event to display travel time markers with name according to phase
    marker_event = clone(event)
    marker_event.time = arrival.t + event.time
    marker_event.name = phase.used_repr()
    marker_event.magnitude = None
    
    markers.append(EventMarker(event=marker_event))
    
    print('{:<12}|{:>11}'.format(phase.used_repr(), util.time_to_str(arrival.t + event.time, format='%H:%M:%S')))

Wow, there are quite some arrivals! Now the questions are:

<div class='alert alert-success'>
    <p style="font-weight: bold; font-size: 150%">Task 3:</p>
    <ol>
        <li>Why do we have so many different P and S-wave arrivals?</li>
        <li>Which arrival has likely produced significant signals within our recorded data?</li>
    </ol>
    <span style='font-weight: bold'>Hints:</span>
    <ul>
        <li>You can get detailed information on each phase, if you print the <em>phase</em> within the code snippet above.</li>
        <li>For direct comparison of the phase arrivals with the observed signal you should start the <em>Snuffler</em> with the recorded signals and the markers. Check <a href='https://pyrocko.org/docs/current/library/reference/trace.html#pyrocko.trace.Trace.snuffle'>the Pyrocko documentation and also module 3 - Introduction to seismological toolboxes.</a>
    </ul>
</div>

In [ ]:
# Import
from pyrocko.squirrel import Squirrel
from pyrocko.trace import snuffle

# Loading of waveforms
sq = Squirrel()

# Add path with data to Squirrel database

# Snuffle traces and markers


<div class='alert alert-success'>
    <p style="font-weight: bold; font-size: 150%">Task 4 (no solutions given):</p>
    <ol>
        <li>Load/make manual picks on the given data and compare them with the ones calculated by cake.</li>
        <li> Are they in good agreement? Why could there be differences?</li>
    </ol>
</div>